# Tryout *google* facets data visualization

- [github source code](https://github.com/pair-code/facets)
- [Installation guide to Jupyter](https://github.com/pair-code/facets#enabling-usage-in-jupyter-notebooks)

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from glob import glob
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

import shutil

# Project
project_common_path = os.path.dirname('.')
project_common_path = os.path.abspath(os.path.join(project_common_path, '..', 'common'))
if not project_common_path in sys.path:
    sys.path.append(project_common_path)

In [3]:
facets_path = os.path.dirname(os.path.join('..', '..', '..', '..'))
facets_path = os.path.abspath(os.path.join(facets_path, 'facets', 'facets_overview', 'python'))
                              
if not facets_path in sys.path:
    sys.path.append(facets_path)

In [4]:
from data_utils import METADATA_CSV, TRAIN_MASKS_CSV, OUTPUT_PATH, GENERATED_DATA

In [5]:
METADATA_CSV.head()

,id,year,make,model,trim1,trim2
0,0004d4463b50,2014.0,Acura,TL,TL,w/SE
1,00087a6bd4dc,2014.0,Acura,RLX,RLX,w/Tech
2,000aa097d423,2012.0,Mazda,MAZDA6,MAZDA6,i Sport
3,000f19f6e7d4,2016.0,Chevrolet,Camaro,Camaro,SS
4,00144e887ae9,2015.0,Acura,TLX,TLX,SH-AWD V6 w/Advance Pkg


## Metadata overview 

In [6]:
from generic_feature_statistics_generator import GenericFeatureStatisticsGenerator
proto = GenericFeatureStatisticsGenerator().ProtoFromDataFrames([{'name': 'Metadata', 'table': METADATA_CSV}])

In [7]:
from IPython.core.display import display, HTML
import base64
protostr = base64.b64encode(proto.SerializeToString()).decode("utf-8")
HTML_TEMPLATE = """<link rel="import" href="/nbextensions/facets-dist/facets-jupyter.html" >
        <facets-overview id="elem"></facets-overview>
        <script>
          document.querySelector("#elem").protoInput = "{protostr}";
        </script>"""
html = HTML_TEMPLATE.format(protostr=protostr)
display(HTML(html))

## Facets dive

Create sprite image (see [this](https://github.com/PAIR-code/facets/tree/master/facets_dive#providing-sprites-for-dive-to-render))

In [ ]:
from data_utils import train_ids
from image_utils import get_image_data
import cv2
import numpy as np

In [ ]:
atlas_url = os.path.abspath(os.path.join(GENERATED_DATA, "complete_atlas_image.jpg"))

_metadata_csv = METADATA_CSV[METADATA_CSV['make'] == 'Acura']

sprit_image_size = (224, 224)
if not os.path.exists(atlas_url):
    ids = _metadata_csv['id'].values
    
    n = 100
    m = int(np.ceil(len(ids) * 1.0 / n))
    complete_image = np.zeros((m*sprit_image_size[0], n*sprit_image_size[1], 3), dtype=np.uint8)
    counter = 0
    for i in range(m):
        ys = i*sprit_image_size[1]
        ye = ys + sprit_image_size[1]
        for j in range(n):
            xs = j*sprit_image_size[0]
            xe = xs + sprit_image_size[0]
            if counter == len(ids):
                break
            image_id = ids[counter] + '_03'; counter+=1
            image_type = 'Train' if image_id in train_ids else 'Test'
            img = get_image_data(image_id, image_type)
            img = cv2.resize(img, dsize=sprit_image_size)            
            complete_image[ys:ye, xs:xe, :] = img[:,:,:]
        if counter == len(ids):
            break        
    
    cv2.imwrite(atlas_url, complete_image)
    
#     atlas_image

In [ ]:
_metadata_csv.head()

In [7]:
# Display the Dive visualization for this data
from IPython.core.display import display, HTML

HTML_TEMPLATE = """<link rel="import" href="/nbextensions/facets-dist/facets-jupyter.html">
        <facets-dive id="elem" height="600"></facets-dive>
        <script>
          var data = {jsonstr};
          document.querySelector("#elem").data = data;
        </script>"""
html = HTML_TEMPLATE.format(jsonstr=METADATA_CSV.to_json(orient='records'))
display(HTML(html))

In [ ]:
# Display the Dive visualization for this data
from IPython.core.display import display, HTML

HTML_TEMPLATE = """<link rel="import" href="/nbextensions/facets-dist/facets-jupyter.html">
        <facets-dive id="elem" height="600"></facets-dive>
        <script>
          var data = {jsonstr};
          var atlas_url = {atlas_url};
          document.querySelector("#elem").data = data;
          // document.querySelector("#elem").atlas-url = "complete_atlas_image.jpg";
        </script>"""
html = HTML_TEMPLATE.format(jsonstr=_metadata_csv.to_json(orient='records'), atlas_url=atlas_url)
display(HTML(html))